In [2]:
import numpy as np

In [87]:
def prior(N,dim):
    return np.random.uniform(-2,2, size=(N,dim))


def simulator(theta, x):
    sigma = np.random.normal(0, 0.1)
    sim = np.dot(theta, x) + sigma
    return sim

def inference(y, x, theta, e):
    accepted_samples = []
    for i in range(1, len(theta)):
        sim = simulator(theta[i, :], x)
        #print(sim)
        dis = np.linalg.norm(sim - y)
        #rint(dis)
        if dis < e:
            accepted_samples.append(theta[i,:])
    return np.array(accepted_samples)


# Define the dimensions and the number of samples
N = 10000
dim =2 # 5, 10

# Observation
y_obs = np.array(0.3)
np.random.seed(42)
x = np.random.uniform(-0.5, 0.5, size=(dim,))
#print(x)
e = 0.1 # threshold

# Sampling thetas
np.random.seed(None)
theta_pr =prior(N, dim)

# List of the accepted samples
samples_pos = inference(y_obs, x, theta_pr, e)

In [88]:
samples_pos.shape # 2: 1107, 5: 956, 10: 685

(685, 10)